In [1]:
import sys
sys.path.insert(1, '/Users/brian/crypto/crocket')

In [2]:
from json import load as json_load
from random import randint, shuffle
from requests import get
from time import sleep, time
from requests import get
from requests_futures.sessions import FuturesSession
from requests.exceptions import ConnectTimeout, ProxyError
from concurrent.futures import as_completed, Future

from crocket.bittrex.bittrex2 import Bittrex, return_request_input

In [3]:
PROXY_LIST_PATH = '/Users/brian/crypto/crocket/proxy_list.txt'
BITTREX_CREDENTIALS_PATH = '/Users/brian/bittrex_credentials.json'

In [4]:
def configure_ip(ip):
    
    return {
        'http': ip,
        'https': ip
    }

def process_response(session, response):
    
    try:
        response.data = response.json()
    except:
        response.data = {
               'success': False,
               'message': 'NO_API_RESPONSE',
               'result': None
            }

In [5]:
# Read files

with open(PROXY_LIST_PATH, 'r') as f:
    proxies = f.read().splitlines()
    
with open(BITTREX_CREDENTIALS_PATH, 'r') as f:
    credentials = json_load(f)

In [6]:
# Create bittrex objects

b1 = Bittrex(api_key=credentials.get('key'), 
             api_secret=credentials.get('secret'), 
             api_version='v1.1')

b2 = Bittrex(api_key=credentials.get('key'), 
             api_secret=credentials.get('secret'), 
             dispatch=return_request_input,
             api_version='v1.1')

In [7]:
# Get list of currencies

response = b1.get_markets()

currencies = [x.get('MarketName') for x in response.get('result') 
              if x.get('BaseCurrency') == 'BTC' and x.get('IsActive')]

markets = currencies

In [8]:
rind = list(range(200))

In [9]:
# Test asynchronous requests

with FuturesSession(max_workers=200) as session:

    for ii in range(0, 1): # TODO: replace with while loop

        shuffle(rind)

        inputs = []
        futures = []

        start = time()

        for index in range(len(markets)):

            request_input = b2.get_market_history(markets[index])
            inputs.append(request_input)

            proxy = configure_ip(proxies[rind[index]])
            print(index, proxy)
            response = session.get(request_input.get('url'), 
                                 background_callback=process_response,
                                 headers={"apisign": request_input.get('apisign')},
                                 timeout=5,
                                 proxies=proxy)

            futures.append([markets[index], response])

        stop = time()
        print('Total elapsed time: {}s'.format(str(stop-start)))


#         for future in as_completed(futures):
#             try:
#                 responses1[index][1] = responses1[index][1].result()
#             except (ProxyError, ConnectTimeout):
#                 print('Failed API call for {}, index: {}.'.format(markets[index], str(index)))

#                 proxy = configure_ip(proxies[rind[index]])
#                 future = get(inputs[index].get('url'), 
#                                  #background_callback=process_response,
#                                  headers={"apisign": inputs[index].get('apisign')},
#                                  timeout=5,
#                                  proxies=proxy)

#                 responses1[index][1] = responses1[index][1].result()


        #print(futures)

        #sleep(30)

0 {'http': '192.126.165.50:8800', 'https': '192.126.165.50:8800'}
1 {'http': '192.126.164.200:8800', 'https': '192.126.164.200:8800'}
2 {'http': '173.234.59.74:8800', 'https': '173.234.59.74:8800'}
3 {'http': '192.126.161.75:8800', 'https': '192.126.161.75:8800'}
4 {'http': '173.234.194.116:8800', 'https': '173.234.194.116:8800'}
5 {'http': '173.208.46.200:8800', 'https': '173.208.46.200:8800'}
6 {'http': '192.126.165.58:8800', 'https': '192.126.165.58:8800'}
7 {'http': '192.126.157.223:8800', 'https': '192.126.157.223:8800'}
8 {'http': '192.126.164.183:8800', 'https': '192.126.164.183:8800'}
9 {'http': '192.126.157.100:8800', 'https': '192.126.157.100:8800'}
10 {'http': '192.126.161.210:8800', 'https': '192.126.161.210:8800'}
11 {'http': '192.161.163.239:8800', 'https': '192.161.163.239:8800'}
12 {'http': '192.126.161.126:8800', 'https': '192.126.161.126:8800'}
13 {'http': '173.234.194.120:8800', 'https': '173.234.194.120:8800'}
14 {'http': '173.234.194.253:8800', 'https': '173.234.19

In [ ]:
for ii in as_completed([x[1] for x in futures]):
    print(ii)

In [12]:
futures[0][1].result()

ProxyError: HTTPSConnectionPool(host='bittrex.com', port=443): Max retries exceeded with url: /api/v1.1/public/getmarkethistory?market=BTC-LTC&marketname=BTC-LTC (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 403 Forbidden',)))

In [ ]:
class FutureRequest(Future):
    
    def __init__(self, future, url, market, apisign=None):
        self.future = future
        self.url = url
        self.market = market
        self.apisign = apisign

In [ ]:
FutureRequest(futures[0][1], 'a', 'b', 'a')

In [ ]:
responses1[125][1].url

In [ ]:
index = 34

proxy = configure_ip(proxies[rind[index]])

future = get(request_input.get('url'), 
                     #background_callback=process_response,
                     headers={"apisign": request_input.get('apisign')},
                     timeout=5,
                     proxies=proxy)

In [ ]:
future

In [ ]:
with FuturesSession(max_workers=200) as session:
    future = session.get(inputs[5].get('url'), 
                                     background_callback=process_response,
                                     headers={"apisign": inputs[5].get('apisign')},
                                     timeout=10,
                                     proxies=configure_ip(proxies[rind[5]]))

In [ ]:
future

In [ ]:
responses1

In [ ]:
import urllib3

In [ ]:
def is_OK(ip):
    try:
        proxy_handler = urllib3.ProxyManager({'http': ip})
        opener = urllib3.build_opener(proxy_handler)
        opener.addheaders = [('User-agent', 'Mozilla/5.0')]
        urllib3.install_opener(opener)
        req=urllib3.Request('http://www.icanhazip.com')
        sock=urllib3.urlopen(req)
    except urllib3.exceptions.HTTPError as e:
        #print 'Error code: ', e.code
        return e.code
    except Exception as detail:

        print(detail)
        return 1
    return 0

In [ ]:
proxies[rind[index]]

In [ ]:
is_OK(proxies[rind[index-2]])

In [ ]:
with open('markets.txt', 'w') as f:
    f.writelines(['{}\n'.format(x) for x in sorted(markets)])